#### Data Analysis

##### Connect to database

In [1]:
import sqlite3
import pandas as pd
conn=sqlite3.connect("retail.db")

#### Basic data check

In [6]:
query="""
select count(*) As total_rows from transactions
"""
pd.read_sql(query,conn)

,total_rows
0,182138


In [7]:
query="""
SELECT name FROM sqlite_master
WHERE type='table';
"""
pd.read_sql(query,conn)

,name
0,transactions


In [8]:
# table structure
query="""
PRAGMA table_info(transactions);
"""
pd.read_sql(query,conn)

,cid,name,type,notnull,dflt_value,pk
0,0,OrderLineID,TEXT,0,None,0
1,1,OrderID,TEXT,0,None,0
2,2,ProductID,TEXT,0,None,0
3,3,Qty,INTEGER,0,None,0
4,4,UnitPrice,REAL,0,None,0
5,5,LineDiscount$,REAL,0,None,0
6,6,LineCost$,REAL,0,None,0
7,7,LineRevenue$,REAL,0,None,0


In [18]:
# df.head()

# Firsst 10 Rows like
query="""
SELECT * FROM  transactions
LIMIT 10;
"""
data=pd.read_sql(query,conn)
data

,OrderLineID,OrderID,ProductID,Qty,UnitPrice,LineDiscount$,LineCost$,LineRevenue$
0,OL0000001,O000001,P001689,2,323.19,45.22,363.40,601.16
1,OL0000002,O000001,P000676,3,34.66,7.27,59.13,96.71
2,OL0000003,O000001,P000972,1,76.08,5.32,45.59,70.76
3,OL0000004,O000001,P001872,3,45.45,9.54,93.42,126.81
4,OL0000005,O000001,P001669,1,78.55,5.50,50.67,73.05
5,OL0000006,O000001,P002043,3,130.43,27.38,201.69,363.91
6,OL0000007,O000002,P003057,4,109.83,92.69,225.28,346.63
7,OL0000008,O000002,P002994,1,261.33,55.14,136.08,206.19
8,OL0000009,O000002,P000980,1,249.56,52.65,156.44,196.91
9,OL0000010,O000002,P001527,1,13.53,2.85,8.81,10.68


In [10]:
# Count total rows

query="""
SELECT COUNT(*) AS total_rows
FROM transactions;
"""
pd.read_sql(query,conn)

,total_rows
0,182138


In [12]:
# Select distinct customers

query="""
select distinct(OrderID) as Customers from transactions;
"""
pd.read_sql(query,conn)

,Customers
0,O000001
1,O000002
2,O000003
3,O000004
4,O000005
...,...
44995,O044996
44996,O044997
44997,O044998
44998,O044999


In [ ]:
# Count customers

query="""
select count(distinct(OrderID)) as total_customers from transactions
where OrderID is not null;
"""
pd.read_sql(query,conn)

# 45000 

,total_customers
0,45000


In [16]:
# Check missing values

query="""
SELECT  COUNT(*) AS missing_customer
FROM transactions
WHERE OrderID IS NULL;
"""
pd.read_sql(query,conn)

,missing_customer
0,0


In [19]:
data

,OrderLineID,OrderID,ProductID,Qty,UnitPrice,LineDiscount$,LineCost$,LineRevenue$
0,OL0000001,O000001,P001689,2,323.19,45.22,363.40,601.16
1,OL0000002,O000001,P000676,3,34.66,7.27,59.13,96.71
2,OL0000003,O000001,P000972,1,76.08,5.32,45.59,70.76
3,OL0000004,O000001,P001872,3,45.45,9.54,93.42,126.81
4,OL0000005,O000001,P001669,1,78.55,5.50,50.67,73.05
5,OL0000006,O000001,P002043,3,130.43,27.38,201.69,363.91
6,OL0000007,O000002,P003057,4,109.83,92.69,225.28,346.63
7,OL0000008,O000002,P002994,1,261.33,55.14,136.08,206.19
8,OL0000009,O000002,P000980,1,249.56,52.65,156.44,196.91
9,OL0000010,O000002,P001527,1,13.53,2.85,8.81,10.68


In [20]:
# Feature engineering in SQL

query="""
select 
Qty,UnitPrice,
Qty* UnitPrice As Revenue
from transactions
LIMIT 10;
"""
pd.read_sql(query,conn)

,Qty,UnitPrice,Revenue
0,2,323.19,646.38
1,3,34.66,103.98
2,1,76.08,76.08
3,3,45.45,136.35
4,1,78.55,78.55
5,3,130.43,391.29
6,4,109.83,439.32
7,1,261.33,261.33
8,1,249.56,249.56
9,1,13.53,13.53


In [21]:
# Total revenue

query="""
select 
    sum(Qty*UnitPrice) As Total_revenue
    from transactions;
"""
pd.read_sql(query,conn)

,Total_revenue
0,55359393.28


In [23]:
# Save table

df_preview=pd.read_sql("SELECT * FROM transactions LIMIT 5;",conn)
df_preview

,OrderLineID,OrderID,ProductID,Qty,UnitPrice,LineDiscount$,LineCost$,LineRevenue$
0,OL0000001,O000001,P001689,2,323.19,45.22,363.40,601.16
1,OL0000002,O000001,P000676,3,34.66,7.27,59.13,96.71
2,OL0000003,O000001,P000972,1,76.08,5.32,45.59,70.76
3,OL0000004,O000001,P001872,3,45.45,9.54,93.42,126.81
4,OL0000005,O000001,P001669,1,78.55,5.50,50.67,73.05


In [ ]:
# revenue generated for each order generated?
query="""
SELECT 
OrderId,
SUM(LineRevenue$) As Order_Revenue,
SUM(Qty) As total_items,
Count(DISTINCT ProductID)
"""

In [ ]:
conn.close()